In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
if 'already_run' not in locals():
    os.chdir("..")
    already_run = True
else:
    print('Already run')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cfg_path = '/home/pica/workspace/fsdl2022/numerai_signals_bot/api/configurations/cfg.yaml'

In [ ]:
path = 'https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_train_val_bbg.csv'
path = '/home/pica/nas_pica/Data/numerai_signals/numerai_files/signals_train_val_bbg.csv'
df = pd.read_csv(path)
df.shape
df['friday_date'] = pd.to_datetime(df['friday_date'], format='%Y%m%d')
df.head()

In [ ]:
per_frid = df.groupby('friday_date')['bloomberg_ticker'].count()

In [ ]:
per_frid.head()
per_frid.plot()

In [ ]:
from src.data.database_generator import DataBaseGenerator

dbg = DataBaseGenerator()
dbg.create_initial_database()

In [ ]:
from src.data.database_reader import DataBaseReader

from pathlib import Path
db_dir = Path('db')
dbr = DataBaseReader(db_dir)

ticker_data = dbr.get_data()

In [ ]:
ticker_data.shape
ticker_data.head()

In [ ]:
import matplotlib.pyplot as plt

usd_idxs = ticker_data.currency == 'USD'
tickers = np.random.choice(ticker_data[usd_idxs]['bloomberg_ticker'].unique(), 5)
tickers
_= plt.figure(figsize=(15,10))
for ticker in tickers:
    idxs = ticker_data['bloomberg_ticker'] == ticker
    _=plt.plot(ticker_data[idxs]['date'], ticker_data[idxs][['high', 'low']].mean(axis=1), label=ticker)
    
plt.legend()
plt.grid()

In [ ]:
from src.features.feature_generator import FeatureGenerator
fg = FeatureGenerator(cfg_path)

fg.generate_features()

In [ ]:
from src.models.model_trainer import ModelTrainer

trainer = ModelTrainer(cfg_path)


In [ ]:
trainer.load_data()

In [ ]:
trainer.train_model()

In [ ]:
trainer.predict(output_dir='output')